In [1]:
import pandas as pd
import numpy as np
import keras

from google.colab import files
uploaded = files.upload()

Saving compresive_strength_concrete.csv to compresive_strength_concrete.csv


In [2]:
concrete_data = pd.read_csv('compresive_strength_concrete.csv')
concrete_data.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.size

9270

In [3]:
concrete_data.isnull().sum()

Cement (component 1)(kg in a m^3 mixture)                0
Blast Furnace Slag (component 2)(kg in a m^3 mixture)    0
Fly Ash (component 3)(kg in a m^3 mixture)               0
Water  (component 4)(kg in a m^3 mixture)                0
Superplasticizer (component 5)(kg in a m^3 mixture)      0
Coarse Aggregate  (component 6)(kg in a m^3 mixture)     0
Fine Aggregate (component 7)(kg in a m^3 mixture)        0
Age (day)                                                0
Concrete compressive strength(MPa, megapascals)          0
dtype: int64

In [8]:
concrete_data = concrete_data.sample(frac=1).reset_index(drop=True)
data = concrete_data.drop('Concrete compressive strength(MPa, megapascals) ',1)
targets = concrete_data.loc[:, 'Concrete compressive strength(MPa, megapascals) ']

data.shape

(1030, 8)

In [9]:
targets.shape

(1030,)

In [12]:
train_data = data.iloc[0:515]
train_targets = targets.iloc[0:515]

val_data = data.iloc[515:721]
val_targets = targets.iloc[515:721]

test_data = data.iloc[721:]
test_targets =targets.iloc[721:]

print('train_data shape:', train_data.shape)
print('train_targets shape:',train_targets.shape)

print('val_data shape:', val_data.shape)
print('val_targets shape:', val_targets.shape)

print('test_data shape:', test_data.shape)
print('test_targets shape:', test_targets.shape)

train_data shape: (515, 8)
train_targets shape: (515,)
val_data shape: (206, 8)
val_targets shape: (206,)
test_data shape: (309, 8)
test_targets shape: (309,)


In [13]:
mean = train_data.mean()
std = train_data.std()

train_data -= mean
train_data /= std

val_data -= mean
val_data /= std

test_data -= mean
test_data /= std

train_data.head(2)

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day)
0,-0.352224,-0.809045,0.715400,-1.116861,1.126318,0.172536,1.436312,-0.650628
1,-0.903960,-0.809045,1.133728,-0.931872,0.633913,1.485170,0.354633,0.840364


In [22]:
from keras import models, layers, regularizers
  
model = models.Sequential()
model.add(layers.Dense(10, activation='relu', input_shape=(train_data.shape[1],), kernel_regularizer=regularizers.L2(0.001)))
model.add(layers.Dense(8, activation='relu', kernel_regularizer=regularizers.L2(0.001)))
model.add(layers.Dense(6, activation='relu', kernel_regularizer=regularizers.L2(0.001)))
model.add(layers.Dense(1))
    
model.compile(optimizer='Adam', loss='mse', metrics=['mae'])

history = model.fit(train_data, train_targets, validation_data = (val_data, val_targets), epochs=100, verbose=1)

Epoch 1/100
17/17 [==============================] - 1s 14ms/step - loss: 1552.8844 - mae: 35.7935 - val_loss: 1661.0819 - val_mae: 37.2014
Epoch 2/100
17/17 [==============================] - 0s 4ms/step - loss: 1441.0781 - mae: 34.1060 - val_loss: 1652.0256 - val_mae: 37.0841
Epoch 3/100
17/17 [==============================] - 0s 4ms/step - loss: 1584.4748 - mae: 35.8766 - val_loss: 1642.2529 - val_mae: 36.9504
Epoch 4/100
17/17 [==============================] - 0s 4ms/step - loss: 1523.1470 - mae: 35.2035 - val_loss: 1628.3258 - val_mae: 36.7641
Epoch 5/100
17/17 [==============================] - 0s 4ms/step - loss: 1570.7264 - mae: 35.9616 - val_loss: 1605.2876 - val_mae: 36.4729
Epoch 6/100
17/17 [==============================] - 0s 4ms/step - loss: 1391.3078 - mae: 33.6294 - val_loss: 1573.2799 - val_mae: 36.0642
Epoch 7/100
17/17 [==============================] - 0s 4ms/step - loss: 1479.1786 - mae: 34.8441 - val_loss: 1531.5073 - val_mae: 35.5293
Epoch 8/100
17/17 [=======

In [24]:
predictions = model.predict(test_data)
predictions.shape

(309, 1)

In [25]:
predictions = predictions.reshape(309)
output = pd.DataFrame({'Actual': test_targets})
output['Predicted'] = predictions
output['Absolute Error'] = np.abs(output['Predicted'] - output['Actual'])
output

,Actual,Predicted,Absolute Error
721,44.64,37.957825,6.682175
722,33.56,38.398220,4.838220
723,19.69,24.279390,4.589390
724,40.86,34.251045,6.608955
725,33.76,47.913082,14.153082
...,...,...,...
1025,45.37,49.579910,4.209910
1026,31.64,29.443262,2.196738
1027,64.30,56.160320,8.139680
1028,41.15,53.539825,12.389825
